In [1]:
import os

os.makedirs("models", exist_ok=True)
os.makedirs("static/uploads", exist_ok=True)
os.makedirs("templates", exist_ok=True)


In [3]:
html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Speech Emotion Recognition</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body { background-color: #f4f4f4; }
        .container { margin-top: 50px; max-width: 600px; }
        .card { border-radius: 20px; }
        .btn-primary { width: 100%; }
        audio { width: 100%; margin-top: 15px; }
        .table { margin-top: 20px; }
    </style>
</head>
<body>
<div class="container">
    <div class="card shadow p-4">
        <h2 class="text-center mb-4">🎤 Speech Emotion Recognition</h2>
        <form method="POST" enctype="multipart/form-data">
            <input class="form-control mb-3" type="file" name="audio" accept=".wav" required>
            <button type="submit" class="btn btn-primary">Predict Emotion</button>
        </form>
        {% if file_path %}
        <div class="mt-4">
            <h5>🔊 Uploaded Audio:</h5>
            <audio controls>
                <source src="{{ file_path }}" type="audio/wav">
                Your browser does not support the audio element.
            </audio>
        </div>
        <div class="mt-4">
            <h5>🎯 Prediction: <strong>{{ prediction }}</strong></h5>
            <p>📈 Confidence: <strong>{{ confidence }}%</strong></p>
        </div>
        {% endif %}
        {% if logs %}
        <div class="mt-4">
            <h5>📜 Prediction Logs</h5>
            <table class="table table-bordered">
                <thead class="table-light">
                    <tr>
                        <th>Filename</th>
                        <th>Emotion</th>
                        <th>Confidence</th>
                    </tr>
                </thead>
                <tbody>
                {% for row in logs %}
                    <tr>
                        <td>{{ row[0] }}</td>
                        <td>{{ row[1] }}</td>
                        <td>{{ row[2] }}%</td>
                    </tr>
                {% endfor %}
                </tbody>
            </table>
        </div>
        {% endif %}
    </div>
</div>
</body>
</html>
"""

# Save it to templates/index.html
with open("templates/index.html", "w", encoding="utf-8") as f:
    f.write(html_code)


In [7]:
from flask import Flask, render_template, request
import os
import numpy as np
import librosa
import soundfile
import joblib
import pandas as pd
from werkzeug.utils import secure_filename

# Create Flask app
app = Flask(__name__)
UPLOAD_FOLDER = "static/uploads"
LOG_FILE = "prediction_logs.csv"

# Load models
model = joblib.load("models/best_model.pkl")
label_encoder = joblib.load("models/label_encoder.pkl")
scaler = joblib.load("models/scaler.pkl")
selector = joblib.load("models/selector.pkl")

# Feature extraction
def extract_features(file_name):
    with soundfile.SoundFile(file_name) as sf:
        X = sf.read(dtype="float32")
        sample_rate = sf.samplerate
        result = np.array([])

        stft = np.abs(librosa.stft(X))

        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))

        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
        result = np.hstack((result, chroma))

        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, mel))

        return result

@app.route("/", methods=["GET", "POST"])
def index():
    prediction, confidence, file_path = None, None, None

    if request.method == "POST":
        file = request.files["audio"]
        if file and file.filename.endswith(".wav"):
            filename = secure_filename(file.filename)
            filepath = os.path.join(UPLOAD_FOLDER, filename)
            file.save(filepath)

            features = extract_features(filepath)
            features = features.reshape(1, -1)
            features = scaler.transform(features)
            features = selector.transform(features)

            pred_idx = model.predict(features)[0]
            prediction = label_encoder.inverse_transform([pred_idx])[0]

            if hasattr(model, "predict_proba"):
                probas = model.predict_proba(features)[0]
                confidence = round(float(probas[pred_idx]) * 100, 2)
            else:
                confidence = None

            file_path = "/" + filepath.replace("\\", "/")

            # Save log
            log_row = pd.DataFrame([[filename, prediction, confidence]], columns=["Filename", "Emotion", "Confidence"])
            if os.path.exists(LOG_FILE):
                existing = pd.read_csv(LOG_FILE)
                log_row = pd.concat([existing, log_row], ignore_index=True)
            log_row.to_csv(LOG_FILE, index=False)

    # Read logs
    logs = []
    if os.path.exists(LOG_FILE):
        df_logs = pd.read_csv(LOG_FILE)
        logs = df_logs.values.tolist()

    return render_template("index.html",
                           prediction=prediction,
                           confidence=confidence,
                           file_path=file_path,
                           logs=logs)

# Start Flask in background from notebook
from threading import Thread

def run_flask():
    app.run(port=5050)

# Launch once
thread = Thread(target=run_flask)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5050
Press CTRL+C to quit
